## importing data

In [1]:
import pandas as pd

In [2]:
love_added = pd.DataFrame.from_csv("data/love.added.csv")
love_deleted = pd.DataFrame.from_csv("data/love.deleted.csv")

wisdom_added = pd.DataFrame.from_csv("data/wisdom.added.csv")
wisdom_deleted = pd.DataFrame.from_csv("data/wisdom.deleted.csv")

morality_added = pd.DataFrame.from_csv("data/morality.added.csv")
morality_deleted = pd.DataFrame.from_csv("data/morality.deleted.csv")

ethics_added = pd.DataFrame.from_csv("data/ethics.added.csv")
ethics_deleted = pd.DataFrame.from_csv("data/ethics.deleted.csv")

In [3]:
love = love_added.join(love_deleted, how="outer", lsuffix=" added", rsuffix=" deleted")
wisdom = wisdom_added.join(wisdom_deleted, how="outer", lsuffix=" added", rsuffix=" deleted")
morality = morality_added.join(morality_deleted, how="outer", lsuffix=" added", rsuffix=" deleted")
ethics = ethics_added.join(ethics_deleted, how="outer", lsuffix=" added", rsuffix=" deleted")

In [22]:
ignore_list = [ "a", "of", "and", "to", "the", "is", "for", "or" , "in", "that", "it", "|", "ref",
               "http", "''", "``", "s", "an", "-", "=", "*", "==", "===", "====", "name=", "nbsp", "style=", "5px",
              "font-siz", "|-"]

love["added - deleted"] = map(lambda x, y: x-y, love["count added"], love["count deleted"])
love["abs(added - deleted)"] = map(lambda x, y: abs(x-y), love["count added"], love["count deleted"])
love = love.drop([ w for w in ignore_list if w in love.index ])
love = love.sort(["abs(added - deleted)"], ascending=[0])

love.head(20)

,count added,inflections added,count deleted,inflections deleted,added - deleted,abs(added - deleted)
love,44834,"Loved, LOve, LOVES, LOVED, lOVES, Loves, love,...",45228,"Loved, LOve, LOVES, LOVED, lOVES, Loves, love,...",-394,394
word,3091,"word, Words, words, WORD, Word",3172,"Word, words, WORD, word",-81,81
be,6379,"be, BE, being, Being, beings, Be, BEING",6451,"be, BE, being, Being, beings, Be, BEING",-72,72
person,3019,"personalities, PERSON, personal, person, Perso...",3082,"personalities, PERSON, personal, person, Perso...",-63,63
with,4672,"withing, with, With, WiTH, WITH",4734,"withing, with, With, WiTH, WITH",-62,62
''love,1739,"''LOVE, ''lovely, ''Love, ''love",1797,"''LOVE, ''lovely, ''love, ''Love",-58,58
desir,1824,"desire, Desire, desires, desirable, desired, d...",1882,"desire, Desire, desires, desirable, desired, d...",-58,58
concept,1360,"conceptions, concept, conception, concepts, Co...",1418,"conceptions, concept, conception, concepts, Co...",-58,58
can,2836,"CAN, Can, can",2892,"CAN, Can, can",-56,56
emot,2898,"emotion, Emotion, emotionally, Emotions, emoti...",2952,"emotion, Emotion, emotionally, Emotions, emoti...",-54,54


In [23]:
wisdom["added - deleted"] = map(lambda x, y: x-y, wisdom["count added"], wisdom["count deleted"])
wisdom["abs(added - deleted)"] = map(lambda x, y: abs(x-y), wisdom["count added"], wisdom["count deleted"])
wisdom = wisdom.drop([ w for w in ignore_list if w in wisdom.index ])
wisdom = wisdom.sort(["abs(added - deleted)"], ascending=[0])

wisdom.head(20)

,count added,inflections added,count deleted,inflections deleted,added - deleted,abs(added - deleted)
wisdom,3727,"wisdom, wisdomic, WISDOM, Wisdom",3884,"wisdom, wisdomic, WISDOM, Wisdom",-157,157
knowledg,968,"knowledge, knowledgeable, knowledged, KNOWLEDG...",1045,"knowledge, knowledgeable, knowledged, KNOWLEDG...",-77,77
with,1315,"with, With, WITH",1364,"with, With, WITH",-49,49
experi,280,"Experiments, Experiences, Experience, experien...",326,"Experiments, Experiences, Experience, experien...",-46,46
by,550,"by, By",592,"by, By",-42,42
tradit,96,"traditional, Traditional, Traditionally, tradi...",127,"traditional, Traditional, Traditionally, tradi...",-31,31
action,768,"actions, action, Actions",798,"actions, action, Actions",-30,30
most,86,"most, Most",114,"most, Most",-28,28
understand,581,"understands, understandings, understanding, Un...",607,"understands, understandings, understanding, un...",-26,26
life,345,"life, Life",371,"life, Life",-26,26


In [24]:
morality["added - deleted"] = map(lambda x, y: x-y, morality["count added"], morality["count deleted"])
morality["abs(added - deleted)"] = map(lambda x, y: abs(x-y), morality["count added"], morality["count deleted"])

morality = morality.drop([ w for w in ignore_list if w in morality.index ])
morality = morality.sort(["abs(added - deleted)"], ascending=[0])

morality.head(20)

,count added,inflections added,count deleted,inflections deleted,added - deleted,abs(added - deleted)
moral,9118,"Morally, morals, morallity, Moralities, Morala...",9352,"Morally, morals, morallity, Moralities, Morala...",-234,234
are,1991,"ARE, are, Are",2079,"ARE, are, Are",-88,88
with,1632,"With, with, WITH",1709,"With, with, WITH",-77,77
be,2322,"be, beings, being, Be, Being",2396,"be, beings, being, Be, Being",-74,74
from,1383,"From, from, FROM",1449,"From, from, FROM",-66,66
on,1835,"on, On, ON",1900,"on, On, ON",-65,65
right,1030,"Right, Rights, rights, righting, right, rightn...",1085,"Right, Rights, rights, righting, right, rightn...",-55,55
code,1421,"code, Code, Codes, codes",1472,"code, Code, Codes, codes",-51,51
such,868,"such, SUCH, Such",917,"such, SUCH, Such",-49,49
ethic,1718,"ethic, ethical, Ethical, ethically, Ethic, eth...",1766,"ethic, ethical, Ethical, ethically, Ethic, eth...",-48,48


In [25]:
ethics["added - deleted"] = map(lambda x, y: x-y, ethics["count added"], ethics["count deleted"])
ethics["abs(added - deleted)"] = map(lambda x, y: abs(x-y), ethics["count added"], ethics["count deleted"])
ethics = ethics.drop([ w for w in ignore_list if w in ethics.index ])
ethics = ethics.sort(["abs(added - deleted)"], ascending=[0])

ethics.head(20)

,count added,inflections added,count deleted,inflections deleted,added - deleted,abs(added - deleted)
ethic,11589,"ethic, ethical, ETHIC, Ethicism, Ethical, ethi...",11848,"ethic, ethical, ETHIC, Ethicism, Ethical, ethi...",-259,259
philosophi,1726,"Philosophy, Philosophies, PHILOSOPHY, philosop...",1862,"Philosophy, Philosophies, PHILOSOPHY, philosop...",-136,136
moral,3476,"morally, morals, Morals, MORALS, moral, Morale...",3586,"morally, morals, Morals, MORALS, Morale, MORAL...",-110,110
what,1695,"what, whats, WHAT, WHATS, What",1805,"What, whats, WHAT, WHATS, what",-110,110
right,2399,"Right, right, rights, rightness, Rights",2488,"Right, Rights, rights, right, rightness",-89,89
from,1329,"From, from",1417,"From, from",-88,88
which,1590,"Which, which",1674,"Which, which",-84,84
valu,978,"valued, Value, valuing, values, value, Values",1059,"valued, Value, valuing, values, value, Values",-81,81
branch,515,"branches, branch, Branch, Branches",595,"branches, branch, Branch, Branches",-80,80
philosoph,1143,"Philosophers, philosophically, philosophizing,...",1221,"Philosophers, philosophically, philosophizing,...",-78,78
